In [19]:
import pandas as pd
import numpy as np

df = pd.read_csv("centersok.csv", encoding="utf-8-sig")

# Show the first few rows
print(df.head())

   center_id                                  descripcio_servei  \
0       9353  Residència assistida per a gent gran - tempora...   
1       9354  Residència assistida per a gent gran - tempora...   
2       9355  Residència assistida per a gent gran - tempora...   
3       9357  Residència assistida per a gent gran - tempora...   
4       9358  Residència assistida per a gent gran - tempora...   

                                                 nom  \
0                                  Residència Canigó   
1                    Residència Germans Aymar i Puig   
2                   Residencia Geriàtrica els Rosers   
3  Residència Francesca Roig (servei de residènci...   
4                                  Residència Millet   

                                              adreça  \
0         C. Menorca, 16 08630 Abrera,Baix Llobregat   
1   Av. Germans Aymar i Puig, 9 08328 Alella,Maresme   
2           Av. dels Rosers, 40 08328 Alella,Maresme   
3  C. Camí de Sant Nicolau, 6 08480 

In [20]:
df = df[df["descripcio_servei"].str.contains("Residència assistida per a gent gran", case=False, na=False)]

print(df.head())

   center_id                                  descripcio_servei  \
0       9353  Residència assistida per a gent gran - tempora...   
1       9354  Residència assistida per a gent gran - tempora...   
2       9355  Residència assistida per a gent gran - tempora...   
3       9357  Residència assistida per a gent gran - tempora...   
4       9358  Residència assistida per a gent gran - tempora...   

                                                 nom  \
0                                  Residència Canigó   
1                    Residència Germans Aymar i Puig   
2                   Residencia Geriàtrica els Rosers   
3  Residència Francesca Roig (servei de residènci...   
4                                  Residència Millet   

                                              adreça  \
0         C. Menorca, 16 08630 Abrera,Baix Llobregat   
1   Av. Germans Aymar i Puig, 9 08328 Alella,Maresme   
2           Av. dels Rosers, 40 08328 Alella,Maresme   
3  C. Camí de Sant Nicolau, 6 08480 

In [21]:
# Rename Catalan headers to English
df = df.rename(
    columns={
        "nom": "name",
        "adreça": "address",
        "titularitat": "ownership",
        "temps_mitja_espera": "mean_waiting_time",
        "places_publiques": "public_places",
        "places_privades": "private_places",
        "places_totals": "total_places", # use “total_places” instead if that was the intent
    },
    errors="raise"        # raises KeyError if any original column is missing
)

# Confirm the change
print(df.columns.tolist())


['center_id', 'descripcio_servei', 'name', 'address', 'ownership', 'mean_waiting_time', 'public_places', 'private_places', 'total_places']


In [30]:
df.describe()


,center_id,public_places,private_places,total_places,waiting_time_months
count,851.000000,710.000000,756.00000,615.000000,689.0
mean,9877.472385,43.160563,33.22619,71.141463,15.859216
std,294.504497,32.561750,34.33347,47.167155,12.647694
min,9353.000000,1.000000,-3.00000,9.000000,0.0
25%,9634.500000,20.000000,11.00000,35.000000,6.0
50%,9883.000000,33.500000,23.00000,58.000000,12.0
75%,10130.500000,58.000000,42.00000,91.000000,23.0
max,10484.000000,222.000000,255.00000,308.000000,70.0


In [31]:
import pandas as pd
import numpy as np

# Step 1: Remove rows that explicitly say there's no access list
df = df[~df["mean_waiting_time"].str.contains("Sense llista d'accés", na=False)].copy()

# Step 2: Function to convert waiting time to months
def convert_waiting_time(text):
    if pd.isna(text):
        return pd.NA

    text = text.strip().lower()

    # Treat "info" or empty as missing
    if text == "info":
        return pd.NA
    if text == "Sense llista d'accés":
        return pd.NA     

    if "més de 3 anys" in text:
        return 36

    if "menys d’1 mes" in text or "menys d'1 mes" in text:
        return 1

    # Clean "info", "i", and extra whitespace
    text = text.replace("info", "").replace("i", "").replace("  ", " ").strip()

    parts = text.split()
    years = 0
    months = 0

    for i in range(len(parts)):
        if parts[i] in ["any", "anys"]:
            try:
                years = int(parts[i - 1])
            except:
                pass
        elif parts[i] in ["mes", "mesos"]:
            try:
                months = int(parts[i - 1])
            except:
                pass

    return years * 12 + months

# Step 3: Apply function to create clean months column
df.loc[:, "waiting_time_months"] = df["mean_waiting_time"].apply(convert_waiting_time).astype("Int64")

# Step 4: Optional preview
print(df[["mean_waiting_time", "waiting_time_months"]].head())


  mean_waiting_time  waiting_time_months
0          10 mesos                   10
1            4 anys                   48
2  2 anys i 2 mesos                   26
3  2 anys i 3 mesos                   27
4             1 any                   12


In [32]:
# Step 3: Apply function to create clean months column
df.loc[:, "waiting_time_months"] = df["mean_waiting_time"].apply(convert_waiting_time).astype("Int64")

# Step 4: Optional preview
print(df[["mean_waiting_time", "waiting_time_months"]].head())


  mean_waiting_time  waiting_time_months
0          10 mesos                   10
1            4 anys                   48
2  2 anys i 2 mesos                   26
3  2 anys i 3 mesos                   27
4             1 any                   12


In [33]:

df.head()

,center_id,descripcio_servei,name,address,ownership,mean_waiting_time,public_places,private_places,total_places,waiting_time_months
0,9353,Residència assistida per a gent gran - tempora...,Residència Canigó,"C. Menorca, 16 08630 Abrera,Baix Llobregat","Canigó serveis geriàtrics, SL",10 mesos,60.0,19.0,79.0,10
1,9354,Residència assistida per a gent gran - tempora...,Residència Germans Aymar i Puig,"Av. Germans Aymar i Puig, 9 08328 Alella,Maresme","Germans Aymar i Puig, Fundació Privada",4 anys,30.0,88.0,118.0,48
2,9355,Residència assistida per a gent gran - tempora...,Residencia Geriàtrica els Rosers,"Av. dels Rosers, 40 08328 Alella,Maresme","Lantus, SL",2 anys i 2 mesos,9.0,26.0,35.0,26
3,9357,Residència assistida per a gent gran - tempora...,Residència Francesca Roig (servei de residènci...,"C. Camí de Sant Nicolau, 6 08480 Ametlla del V...",Fundació Antònia Roura Barbany,2 anys i 3 mesos,27.0,30.0,57.0,27
4,9358,Residència assistida per a gent gran - tempora...,Residència Millet,"C. Camí de Puiggraciós, 1 08480 Ametlla del Va...","Millet Park, SL",1 any,28.0,18.0,46.0,12


In [34]:
# Drop rows with NaN waiting times to avoid errors
max_waiting_row = df.dropna(subset=["waiting_time_months"]).sort_values("waiting_time_months", ascending=False).iloc[0]
print("1. Center with the biggest waiting time:")
print(max_waiting_row[["name", "waiting_time_months"]])


1. Center with the biggest waiting time:
name                   Residència de la Fundació Privada Asil Inglada...
waiting_time_months                                                   70
Name: 588, dtype: object


In [35]:
min_waiting_row = df.dropna(subset=["waiting_time_months"]).sort_values("waiting_time_months", ascending=True).iloc[0]
print("\n2. Center with the shortest waiting time:")
print(min_waiting_row[["name", "waiting_time_months"]])



2. Center with the shortest waiting time:
name                   Residència geriàtrica Pla d'Urgell
waiting_time_months                                     1
Name: 695, dtype: object


In [38]:
# Step 1: Find the minimum waiting time
min_wait = df["waiting_time_months"].min()

# Step 2: Get all centers with that waiting time
shortest_wait_df = df[df["waiting_time_months"] == min_wait]

# Step 3: Show the results
print(shortest_wait_df)


     center_id                                  descripcio_servei  \
80        9457  Residència assistida per a gent gran - tempora...   
695      10192  Residència assistida per a gent gran - tempora...   
732      10230  Residència assistida per a gent gran - tempora...   
742      10240  Residència assistida per a gent gran - tempora...   
822      10326  Residència assistida per a gent gran - tempora...   

                                                  name  \
80                           IPF Servicios Geriátricos   
695                 Residència geriàtrica Pla d'Urgell   
732                       Centre geriàtric del Pirineu   
742  Residència per a Gent Gran Serafí Casanovas Sansa   
822                               Residència Vandellós   

                                               address  \
80     C. Caspe, 82, 4rt 2ª 08010 Barcelona,Barcelonès   
695  Ctr. Nacional II, núm. 3 25220 Bell-lloc d'Urg...   
732  C. Indústria, 3 25500 Pobla de Segur, la,Palla...   
742 

In [41]:
# Find the maximum value, ignoring NaNs
max_public = df["public_places"].max()

# Show the result
print("Maximum public places:", max_public)

# Filter centers with the maximum number of public places
most_public_df = df[df["public_places"] == max_public]

# Show the result
print(most_public_df)

Maximum public places: 222.0
     center_id                                  descripcio_servei  \
632      10124  Residència assistida per a gent gran - tempora...   

                                         name  \
632  Residència per a Gent Gran Creu de Palau   

                                      address                 ownership  \
632  C. Jocs Olímpics, 2 17003 Girona,Gironès  Generalitat de Catalunya   

    mean_waiting_time  public_places  private_places  total_places  \
632          10 mesos          222.0             NaN           NaN   

     waiting_time_months  
632                   10  


In [14]:
# Find the maximum value in public_places (ignoring NaNs)
max_public_places = df["public_places"].max()

# Filter centers with that number of public places
centers_with_max_public = df[df["public_places"] == max_public_places]

# Get the one with the highest waiting time among them
row = centers_with_max_public.dropna(subset=["waiting_time_months"]).sort_values("waiting_time_months", ascending=False).iloc[0]

print("\n3. Center with max public places and biggest waiting time:")
print(row[["name", "public_places", "waiting_time_months", "address"]])




3. Center with max public places and biggest waiting time:
name                   Residència per a Gent Gran Creu de Palau
public_places                                             222.0
waiting_time_months                                          10
address                C. Jocs Olímpics, 2 17003 Girona,Gironès
Name: 632, dtype: object


In [42]:
# Step 1: Find the maximum number of private places
max_private = df["private_places"].max()

# Step 2: Filter only the rows with that maximum
max_private_df = df[df["private_places"] == max_private]

# Step 3: From that subset, find the longest waiting time
max_wait = max_private_df["waiting_time_months"].max()

# Step 4: Get the center(s) that match both
result = max_private_df[max_private_df["waiting_time_months"] == max_wait]

# Show the result
print(result)

    center_id                                  descripcio_servei  \
98       9485  Residència assistida per a gent gran - tempora...   

                             name  \
98  Sanitas Residencial Les Corts   

                                              address            ownership  \
98  C. Evarist Arnús, 22-32 08014 Barcelona,Barcel...  Sanitas Mayores, SL   

   mean_waiting_time  public_places  private_places  total_places  \
98  4 anys i 2 mesos           53.0           255.0         308.0   

    waiting_time_months  
98                   50  


In [43]:

# Step 1: Filter centers with "Barcelona" in the address (case-insensitive)
barcelona_df = df[df["address"].str.contains("Barcelona", case=False, na=False)]

# Step 2: Drop rows with missing waiting_time_months
barcelona_df = barcelona_df.dropna(subset=["waiting_time_months"])

# Step 3: Find the maximum waiting time in Barcelona
max_wait = barcelona_df["waiting_time_months"].max()

# Step 4: Get center(s) that match the max
result = barcelona_df[barcelona_df["waiting_time_months"] == max_wait]

# Show the result
print(result)

    center_id                                  descripcio_servei  \
94       9481  Residència assistida per a gent gran - tempora...   

                                   name  \
94  Sanitas Residencial Sagrada Familia   

                                              address            ownership  \
94  C. Castillejos, 256-258 08013 Barcelona,Barcel...  Sanitas Mayores, SL   

   mean_waiting_time  public_places  private_places  total_places  \
94  5 anys i 7 mesos           15.0           133.0         148.0   

    waiting_time_months  
94                   67  


In [46]:
# Step 1: Filter for centers in Barcelona
barcelona_df = df[df["address"].str.contains("Barcelona", case=False, na=False)]

# Step 2: Drop rows with missing public_places or waiting_time_months
barcelona_df = barcelona_df.dropna(subset=["public_places", "waiting_time_months"])

# Step 3: Find the maximum number of public places
max_public = barcelona_df["public_places"].max()

# Step 4: Filter centers with that maximum
max_public_df = barcelona_df[barcelona_df["public_places"] == max_public]

# Step 5: Find the longest waiting time among them
max_wait = max_public_df["waiting_time_months"].max()

# Step 6: Get the center(s) with both max public places and longest wait
result = max_public_df[max_public_df["waiting_time_months"] == max_wait]

# Show result
print(result)


    center_id                                  descripcio_servei  \
93       9480  Residència assistida per a gent gran - tempora...   

                              name  \
93  Residència Municipal Fortpienc   

                                            address                ownership  \
93  C. Sardenya, 139-147 08013 Barcelona,Barcelonès  Ajuntament de Barcelona   

   mean_waiting_time  public_places  private_places  total_places  \
93          11 mesos          130.0             7.0         137.0   

    waiting_time_months  
93                   11  
